# Setup

**Step 1**: Import Semantic Kernel SDK from pypi.org

In [ ]:
!python -m pip install semantic-kernel==0.5.1.dev0

In [ ]:
import semantic_kernel as sk

kernel = sk.Kernel()

### Configure the service you'd like to use via the `Service` Enum.

In [ ]:
from services import Service

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = Service.AzureOpenAI

## Option 1: using OpenAI

**Step 2**: Add your [OpenAI Key](https://openai.com/product/) key to a `.env` file in the same folder (org Id only if you have multiple orgs):

```
OPENAI_API_KEY="sk-..."
OPENAI_ORG_ID=""
```

Use "keyword arguments" to instantiate an OpenAI Chat Completion service and add it to the kernel:

## Option 2: using Azure OpenAI

**Step 2**: Add your [Azure Open AI Service key](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio) settings to a `.env` file in the same folder:

```
AZURE_OPENAI_API_KEY="..."
AZURE_OPENAI_ENDPOINT="https://..."
AZURE_OPENAI_DEPLOYMENT_NAME="..."
```

Use "keyword arguments" to instantiate an Azure OpenAI Chat Completion service and add it to the kernel:

In [ ]:
service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    api_key, org_id = sk.openai_settings_from_dot_env()
    service_id = "chat-gpt"
    kernel.add_service(
        OpenAIChatCompletion(service_id=service_id, ai_model_id="gpt-3.5-turbo-1106", api_key=api_key, org_id=org_id),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    service_id = "chat_completion"
    kernel.add_service(
        AzureChatCompletion(service_id=service_id, deployment_name=deployment, endpoint=endpoint, api_key=api_key),
    )

# Run a Semantic Function

**Step 3**: Load a Plugin and run a semantic function:

In [ ]:
try:
    plugin = kernel.import_plugin_from_prompt_directory(service_id, "../../samples/plugins", "FunPlugin")
except ValueError as e:
    # Don't fail if we try to add the plug in again
    # This is just for the sake of the example
    # Once the plugin has been added to the kernel it will fail
    # to add it again, if this cell is run multiple times without
    # restarting the kernel
    pass

In [ ]:
joke_function = plugin["Joke"]

joke = await kernel.invoke(joke_function, sk.KernelArguments(input="time travel to dinosaur age", style="super silly"))
print(joke)